Install dependencies.

In [19]:
%pip install -q -U -r requirements.txt
# The options `-q`, `-U`, `-r` respectively refers to 'quiet', 'upgrade', 'requirement'

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


Note: you may need to restart the kernel to use updated packages.


In [2]:
from datasets import load_dataset  # HuggingFace dataset loading function

dataset = load_dataset("allganize/RAG-Evaluation-Dataset-KO")

View the dataset how it is composed.

In [3]:
dataset

DatasetDict({
    test: Dataset({
        features: ['domain', 'question', 'target_answer', 'target_file_name', 'target_page_no', 'context_type', 'alli_gpt-4-turbo_answer', 'alli_gpt-4-turbo_is_correct', 'alli_gpt-4_answer', 'alli_gpt-4_is_correct', 'alli_claude3-opus_answer', 'alli_claude3-opus_is_correct', 'alli_eeve_answer', 'alli_eeve_is_correct', 'alli_llama3_answer', 'alli_llama3_is_correct', 'langchain_gpt-4-turbo_answer', 'langchain_gpt-4-turbo_is_correct', 'langchain_gpt-3.5-turbo_answer', 'langchain_gpt-3.5-turbo_is_correct', 'openai_assistant_gpt-4-turbo_answer', 'openai_assistant_gpt-4-turbo_is_correct', 'openai_assistant_gpt-4_answer', 'openai_assistant_gpt-4_is_correct', 'cohere_command-r_answer', 'cohere_command-r_is_correct', 'cohere_command-r-plus_answer', 'cohere_command-r-plus_is_correct', 'anything_llm_gpt-4-turbo_answer', 'anything_llm_gpt-4-turbo_is_correct', 'anything_llm_gpt-3.5-turbo_answer', 'anything_llm_gpt-3.5-turbo_is_correct', 'anything_llm_claude3-opus_a

In [4]:
df_pandas = dataset['test'].to_pandas()
documents = df_pandas['target_answer'].to_list()

In [5]:
ids = list(range(1,len(documents)+1))

---

Before the steps below, Qdrant must be running in the background.
</br>This could be acheived by executing the following shell command.[^1]

```shell
docker run -p 6333:6333 -p 6334:6334 \
   -v $(pwd)/qdrant_storage:/qdrant/storage:z \
   qdrant/qdrant
```

[^1]: https://qdrant.tech/documentation/quick-start/#download-and-run

In [6]:
from qdrant_client import QdrantClient
client = QdrantClient("localhost", port=6333)

In [7]:
client.delete_collection(collection_name= "general_finance_service")
client.add(collection_name= "general_finance_service",
           documents= documents,
           ids= ids)

[1,
 2,
 3,
 4,
 5,
 6,
 7,
 8,
 9,
 10,
 11,
 12,
 13,
 14,
 15,
 16,
 17,
 18,
 19,
 20,
 21,
 22,
 23,
 24,
 25,
 26,
 27,
 28,
 29,
 30,
 31,
 32,
 33,
 34,
 35,
 36,
 37,
 38,
 39,
 40,
 41,
 42,
 43,
 44,
 45,
 46,
 47,
 48,
 49,
 50,
 51,
 52,
 53,
 54,
 55,
 56,
 57,
 58,
 59,
 60,
 61,
 62,
 63,
 64,
 65,
 66,
 67,
 68,
 69,
 70,
 71,
 72,
 73,
 74,
 75,
 76,
 77,
 78,
 79,
 80,
 81,
 82,
 83,
 84,
 85,
 86,
 87,
 88,
 89,
 90,
 91,
 92,
 93,
 94,
 95,
 96,
 97,
 98,
 99,
 100,
 101,
 102,
 103,
 104,
 105,
 106,
 107,
 108,
 109,
 110,
 111,
 112,
 113,
 114,
 115,
 116,
 117,
 118,
 119,
 120,
 121,
 122,
 123,
 124,
 125,
 126,
 127,
 128,
 129,
 130,
 131,
 132,
 133,
 134,
 135,
 136,
 137,
 138,
 139,
 140,
 141,
 142,
 143,
 144,
 145,
 146,
 147,
 148,
 149,
 150,
 151,
 152,
 153,
 154,
 155,
 156,
 157,
 158,
 159,
 160,
 161,
 162,
 163,
 164,
 165,
 166,
 167,
 168,
 169,
 170,
 171,
 172,
 173,
 174,
 175,
 176,
 177,
 178,
 179,
 180,
 181,
 182,
 183,
 184,
 185

In [20]:
import dspy
from dotenv import load_dotenv
from os import environ

load_dotenv()  # load the `.env` file

# Configure language model
llm = dspy.Google(
    api_key=environ.get('API_KEY')  # Gemini API Key
)

In [8]:
from dspy.retrieve.qdrant_rm import QdrantRM

qdrant_retriever_model = QdrantRM("general_finance_service", client, k=10)

dspy.settings.configure(lm=llm, rm=qdrant_retriever_model)

In [ ]:
# client.set_model("intfloat/multilingual-e5-large") # not tested yet
# ※ https://qdrant.github.io/fastembed/examples/Supported_Models/#supported-text-embedding-models

In [9]:
class RAG(dspy.Module):
    def __init__(self, num_passages=3):
        super().__init__()

        self.retrieve = dspy.Retrieve(k=num_passages)
        self.generate_answer = dspy.ChainOfThought("context, question -> answer")
    
    def forward(self, question):
        context = self.retrieve(question).passages
        prediction = self.generate_answer(context=context, question=question)
        return dspy.Prediction(context=context, answer=prediction.answer)

In [10]:
uncompiled_rag = RAG()

query = "2010년 미국에서 50세 미만에서 발생한 직장암과 결장암의 퍼센트는 어떻게 되며, 이 비율이 2030년에는 어떻게 변할 것으로 예상되는가?"

response = uncompiled_rag(query)

print(response.answer)

In 2010, 5% of colon cancer and 9% of rectal cancer in the United States occurred in people under 50 years of age. According to recent trends, these percentages are expected to increase to 11% of all colon cancer patients and 22% of rectal cancer patients by 2030.
